<a href="https://colab.research.google.com/github/kyt4n/Malaysia-COVID-19-Dataset/blob/main/initial%20bulk%20data%20extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import tweepy
import pandas as pd
import re

## Read allmytweets txt data (not used)


In [ ]:
tweets_df = pd.read_csv('allmytweets.txt', encoding = 'utf-8', header = None, delimiter = ";;;;;;")
tweets_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,0
0,Tahun 2020 diwarnai pandemik virus COVID-19 ya...
1,Sorotan COVID-19 Sepanjang 2020 dari Ketua Pen...
2,Kenyataan Akhbar KPK 31 Disember 2020 – Situas...
3,COVID-19; 31 Dis 2020 Jumlah sembuh baru:1481 ...
4,Dugaan dan cabaran 2020 berperang memutuskan r...
...,...
3221,RT @AdrianGelb: Thrilled by the exciting discu...
3222,RT @WHOWPRO: Q: What is a #coronavirus? A: Cor...
3223,RT @bernamadotcom: #5G jana pendidikan berkual...
3224,RT @WHO: #China🇨🇳 has reported to WHO 139 new ...


In [ ]:
tweets_df

,0
0,Tahun 2020 diwarnai pandemik virus COVID-19 ya...
1,Sorotan COVID-19 Sepanjang 2020 dari Ketua Pen...
2,Kenyataan Akhbar KPK 31 Disember 2020 – Situas...
3,COVID-19; 31 Dis 2020 Jumlah sembuh baru:1481 ...
4,Dugaan dan cabaran 2020 berperang memutuskan r...
...,...
3221,RT @AdrianGelb: Thrilled by the exciting discu...
3222,RT @WHOWPRO: Q: What is a #coronavirus? A: Cor...
3223,RT @bernamadotcom: #5G jana pendidikan berkual...
3224,RT @WHO: #China🇨🇳 has reported to WHO 139 new ...


In [ ]:
# tweets_df.columns.values[0] = "text"
# tweets_df.rename(columns={tweets_df.columns[0]: 'text'})
tweets_df.columns = ['text']
tweets_df

,text
0,Tahun 2020 diwarnai pandemik virus COVID-19 ya...
1,Sorotan COVID-19 Sepanjang 2020 dari Ketua Pen...
2,Kenyataan Akhbar KPK 31 Disember 2020 – Situas...
3,COVID-19; 31 Dis 2020 Jumlah sembuh baru:1481 ...
4,Dugaan dan cabaran 2020 berperang memutuskan r...
...,...
3221,RT @AdrianGelb: Thrilled by the exciting discu...
3222,RT @WHOWPRO: Q: What is a #coronavirus? A: Cor...
3223,RT @bernamadotcom: #5G jana pendidikan berkual...
3224,RT @WHO: #China🇨🇳 has reported to WHO 139 new ...


In [ ]:
bool_rel_idx = tweets_df.text.str.contains('terkini covid|^COVID-19[:;]', regex=True, case=False)
bool_nonrel_idx = tweets_df.text.str.contains('negeri|kluster|vaccin|vaksin', regex=True, case=False)
tweets_df = tweets_df[bool_rel_idx]
tweets_df = tweets_df.loc[~bool_nonrel_idx, :]    # without copying   # ~ invert boolean, subset exclude [non relevant rows idx] [false]

tweets_df = tweets_df[::-1].reset_index(drop=True)     # reorder rows  # reset index
tweets_df

,text
0,COVID-19; Here are the latest updates as of 3p...
1,COVID-19; Just when we thought we have dischar...
2,COVID-19; Nasihat dan kesesuaian mengadakan ma...
3,COVID-19; Home Surveillance shall be done for ...
4,"Status terkini COVID-19, 10 Mac 2020 sehingga ..."
...,...
247,COVID-19; 27 Dis 2020 Jumlah sembuh baru:997 J...
248,COVID-19; 28 Dis 2020 Jumlah sembuh baru:1181 ...
249,COVID-19; 29 Dis 2020 Jumlah sembuh baru:1123 ...
250,COVID-19; 30 Dis 2020 Jumlah sembuh baru:745 J...


In [ ]:
tweets_df.loc[2,'text']

'COVID-19; Nasihat dan kesesuaian mengadakan majlis, persidangan atau perhimpunan orang ramai: https://t.co/oHMUrK2q0i Mar 07, 2020'

In [ ]:
tweets_df.to_csv(r'tweets_df1.csv', index = False)

## Read tweets_df2 csv data

In [2]:
tweets_df = pd.read_csv('tweets_df2.csv')
tweets_df['Date'] = pd.to_datetime(tweets_df['Date'])    # loaded date in string format, change to date format
tweets_df

,Date,tweet_id,text,link
0,2020-09-07,1302890281642074112,"Terkini COVID-19, 7 Sept 2020 Jumlah discaj:9...",t.co/k4C9NzPuRt
1,2020-09-08,1303273872012140544,"Terkini COVID-19, 8 Sep 2020 Jumlah discaj ba...",t.co/xsTCM7mYCE
2,2020-09-09,1303621756515614721,Terkini COVID-19 setakat 9 September 2020 Jum...,t.co/bEuxIL2va5
3,2020-09-11,1304350471872684032,"Terkini COVID-19, 11 Sept 2020 Jumlah discaj ...",t.co/MA0t8Xdd9L
4,2020-09-12,1304706214794723329,"Terkini COVID-19, 12 Sept 2020 Jumlah discaj ...",t.co/PxmVzcehaZ
...,...,...,...,...
234,2021-05-18,1394598840448086018,"COVID-19: 18 Mei 2021 J kes sembuh: 3,497 J k...",t.co/81cOS3A5fT
235,2021-05-19,1394951083114434561,"COVID-19: 19 Mei 2021 J kes sembuh: 3,516 J k...",NaN
236,2021-05-20,1395326594042896392,"COVID-19: 20 Mei 2021 J kes sembuh: 3,916 J k...",t.co/4zEFX1M1kq
237,2021-05-21,1395682030369734657,"COVID-19: 21 Mei 2021 J kes sembuh: 4,508 J k...",t.co/MfKfWXS0bc


In [3]:
fail_rows = []

for i in range(len(tweets_df)):
  text = tweets_df.loc[i,'text']

  try: 
    localcase = re.search(r"(local|lokal|tempatan)(.*?)(imp)", text, re.IGNORECASE).group(0)
    importcase = re.search(r"(import)(.*?)(J)", text, re.IGNORECASE).group(0)
    kematiancase = re.search(r"(mati|fataliti)(.*?)(J)", text, re.IGNORECASE).group(0) 
  except AttributeError:
    fail_rows.append(i)

In [4]:
fail_rows

[38, 135]

In [5]:
tweets_df.loc[38,'text']

'Status terkini COVID-19 setakat 20 Oktober 2020  Jumlah discaj baru: 634 Jumlah kumulatif: 14351 (64.6%)  Jumlah kes baru: 862 Jumlah kumulatif: 22225 Jumlah kes aktif: 7681 Kes import: 2 (1 WN, 1 BWN)  Jumlah kematian: 3 Jumlah kumulatif: 193 (0.87%)  Kes di ICU:95 intubated: 29'

In [6]:
tweets_df.iloc[fail_rows]

,Date,tweet_id,text,link
38,2020-10-20,1318506612961599488,Status terkini COVID-19 setakat 20 Oktober 202...,NaN
135,2021-02-04,1357277748188782593,"COVID-19; 4 Feb 2021 J kes sembuh:4,092 J kum...",t.co/YW0I7RCMhk


In [7]:
tweets_df.drop(fail_rows, inplace=True)
tweets_df.reset_index(drop=True, inplace=True)
tweets_df

,Date,tweet_id,text,link
0,2020-09-07,1302890281642074112,"Terkini COVID-19, 7 Sept 2020 Jumlah discaj:9...",t.co/k4C9NzPuRt
1,2020-09-08,1303273872012140544,"Terkini COVID-19, 8 Sep 2020 Jumlah discaj ba...",t.co/xsTCM7mYCE
2,2020-09-09,1303621756515614721,Terkini COVID-19 setakat 9 September 2020 Jum...,t.co/bEuxIL2va5
3,2020-09-11,1304350471872684032,"Terkini COVID-19, 11 Sept 2020 Jumlah discaj ...",t.co/MA0t8Xdd9L
4,2020-09-12,1304706214794723329,"Terkini COVID-19, 12 Sept 2020 Jumlah discaj ...",t.co/PxmVzcehaZ
...,...,...,...,...
232,2021-05-18,1394598840448086018,"COVID-19: 18 Mei 2021 J kes sembuh: 3,497 J k...",t.co/81cOS3A5fT
233,2021-05-19,1394951083114434561,"COVID-19: 19 Mei 2021 J kes sembuh: 3,516 J k...",NaN
234,2021-05-20,1395326594042896392,"COVID-19: 20 Mei 2021 J kes sembuh: 3,916 J k...",t.co/4zEFX1M1kq
235,2021-05-21,1395682030369734657,"COVID-19: 21 Mei 2021 J kes sembuh: 4,508 J k...",t.co/MfKfWXS0bc


In [9]:
lst_df = []

for i in range(len(tweets_df)):
  text = tweets_df.loc[i,'text']
  
  lst_tweet = re.findall(r"[0-9]+[,.]?[0-9]*[,.]?[0-9]*", text)
  lst_col0_5 = lst_tweet[3:9]       # elemnt 4 until 9  # total 6 
  lst_col17_20 = lst_tweet[-4:]

  localcase = re.search(r"(local|lokal|tempatan)(.*?)(imp)", text, re.IGNORECASE).group(0)      # we only know something is in it
  local_total = re.search(r"[0-9]+[,.]?[0-9]*", localcase).group(0)                             # get the first number  # total 
  if local_total != '0':                                                                        # if not 0 
    local_wn_re = re.search(r"([0-9]+[,.]?[0-9]*)(?:\s?)(?:WN)", localcase, re.IGNORECASE)      # get WN value
    local_bwn_re = re.search(r"([0-9]+[,.]?[0-9]*)(?:\s?)(?:BWN)", localcase, re.IGNORECASE)    # get BWN value
    if local_wn_re is not None:                                                                 # if wn exist
      local_wn = local_wn_re.group(1)                                                           # assign
      if local_bwn_re is None:                                                                  # if either exist but not both, do compute
        local_total = local_total.replace(',' , '')                                             # remove , 
        local_wn = local_wn.replace(',' , '')
        local_bwn = str(int(local_total) - int(local_wn))
    if local_bwn_re is not None:                                                                # if bwn exist
      local_bwn = local_bwn_re.group(1)                                                         # assign
      if local_wn_re is None:                                                                   # if either exist but not both, do compute
        local_total = local_total.replace(',' , '')                                             # remove , 
        local_wn = local_wn.replace(',' , '')
        local_wn = str(int(local_total) - int(local_bwn)) 
    if local_wn_re == local_bwn_re == None:                                                     # if both dont exist, assign empty string
      local_wn = local_bwn = '' 
  else: local_total = local_wn = local_bwn = '0'


  importcase = re.search(r"(import)(.*?)(J)", text, re.IGNORECASE).group(0)                     # we only know something is in it
  import_total = re.search(r"[0-9]+", importcase).group(0)                                      # get the first number  # total 
  if import_total != '0':                                                                       # if not 0 
    import_wn_re = re.search(r"([0-9]+)(?:\s?)(?:WN)", importcase, re.IGNORECASE)               
    import_bwn_re = re.search(r"([0-9]+)(?:\s?)(?:BWN)", importcase, re.IGNORECASE)             
    if import_wn_re is not None:                                                                 
      import_wn = import_wn_re.group(1)
      if import_bwn_re is None:
        import_bwn = str(int(import_total) - int(import_wn))
    if import_bwn_re is not None:
      import_bwn = import_bwn_re.group(1)
      if import_wn_re is None:
        import_wn = str(int(import_total) - int(import_bwn)) 
    if import_wn_re == import_bwn_re == None:                                                   
      import_wn = import_bwn = '' 
  else: import_total = import_wn = import_bwn = '0'


  maticase = re.search(r"(mati|fataliti)(.*?)(J)", text, re.IGNORECASE).group(0)                 # we only know something is in it
  mati_total = re.search(r"[0-9]+", maticase).group(0)                                           # get the first number  # total 
  if import_total != '0':                                                                        # if not 0 
    mati_wn_re = re.search(r"([0-9]+)(?:\s?)(?:WN)", maticase, re.IGNORECASE)
    mati_bwn_re = re.search(r"([0-9]+)(?:\s?)(?:BWN)", maticase, re.IGNORECASE)
    if mati_wn_re is not None:
      mati_wn = mati_wn_re.group(1)
      if mati_bwn_re is None:
        mati_bwn = str(int(mati_total) - int(mati_wn))
    if mati_bwn_re is not None:
      mati_bwn = mati_bwn_re.group(1)
      if mati_wn_re is None:
        mati_wn = str(int(mati_total) - int(mati_bwn)) 
    if mati_wn_re == mati_bwn_re == None:
      mati_wn = mati_bwn = '' 
  else: mati_total = mati_wn = mati_bwn = '0'

  lst_col6_15 = [local_total, local_wn, local_bwn, import_total, import_wn, import_bwn, mati_total, mati_wn, mati_bwn]

  lst_cols = lst_col0_5 + lst_col6_15 + lst_col17_20
  lst_df.append(lst_cols)

In [13]:
cols = ['recovered', 'cumul_recovered', 'cumul_recovered_perc', 'case', 'cumul_case', 
        'case_active', 'case_local', 'case_local_wn', 'case_local_bwn', 'case_import', 
        'case_import_wn', 'case_import_bwn', 'death', 'death_wn', 'death_bwn', 
        'cumul_death', 'cumul_death_perc', 'icu', 'intubated']

df = pd.DataFrame(lst_df, columns=cols)      # 19 cols passed
df.insert(0, 'date', tweets_df['Date'])      # + 1 col date
df['tweet_id'] = tweets_df['tweet_id']       # + 1 col tweet_id
df['link'] = tweets_df['link']               # + 1 col link
df.to_csv(r'tweets_df2_result.csv', index = False)

## Latest bulk data extraction

In [15]:
with open("twitter_creds.json", "r") as file:
  creds = json.load(file)

auth = tweepy.OAuthHandler(creds['api_key'], creds['api_sec'])
auth.set_access_token(creds['access_token'], creds['access_sec'])
api = tweepy.API(auth, wait_on_rate_limit=True)

username = '@DGHisham'
count = 3600

tweets = tweepy.Cursor(api.user_timeline,id=username, tweet_mode="extended", 
                       include_rts=False, exclude_replies=True).items(count)
tweets_list = [[tweet.created_at, tweet.id, tweet.full_text.replace('\n', ' ')] for tweet in tweets]

cols = "date tweet_id text".split()
tweets_df = pd.DataFrame(tweets_list, columns=cols)

In [19]:
bool_rel_idx = tweets_df.text.str.contains('terkini covid|^COVID-19[:;]', regex=True, case=False)
bool_nonrel_idx = tweets_df.text.str.contains('negeri|kluster|vaccin|vaksin', regex=True, case=False)
tweets_df = tweets_df[bool_rel_idx]
tweets_df = tweets_df.loc[~bool_nonrel_idx, :]    # without copying   # ~ invert boolean, subset exclude [non relevant rows idx] [false]

tweets_df = tweets_df[::-1].reset_index(drop=True)     # reorder rows, reset index
tweets_df['date'] = pd.to_datetime(tweets_df['date'].dt.date)
tweets_df[['text','link']] = tweets_df['text'].str.split('https://', expand=True)

In [20]:
tweets_df

,date,tweet_id,text,link
0,2021-02-15,1361250755588485123,"COVID-19; 15 Feb 2021 J kes sembuh: 4,521 J k...",t.co/Yhr88R9Shv
1,2021-02-16,1361632748994060288,"COVID-19; 16 Feb 2021 J kes sembuh: 5,718 J k...",t.co/INHqORa9ja
2,2021-02-17,1361980739235893251,"COVID-19; 17 Feb 2021 J kes sembuh: 5,709 J k...",t.co/rLeKuYlA6L
3,2021-02-18,1362345745727315969,"COVID-19; 18 Feb 2021 J kes sembuh:5,320 J ku...",t.co/eYXvcZ25Fb
4,2021-02-19,1362703258456711169,"COVID-19; 19 Feb 2021 J kes sembuh:4,889 J ku...",t.co/d7BDBLGSTv
...,...,...,...,...
153,2021-07-21,1417798072185212929,"COVID-19: 21 Julai 2021 J kes sembuh:7,902 J ...",t.co/wVhqfvmO24
154,2021-07-22,1418169018712686595,"COVID-19: 22 Julai 2021 J kes sembuh: 8,436 J...",t.co/nn6g2PySnt
155,2021-07-23,1418525491049361411,"COVID-19: 23 Julai 2021 J kes sembuh:10,094 J...",t.co/94rlWVXKBm
156,2021-07-24,1418886465052946434,"COVID-19: 24 Julai 2021 J kes sembuh:9,471 J ...",t.co/joYSRdlcF7


In [23]:
tweets_df.loc[2,'text']

'COVID-19; 17 Feb 2021  J kes sembuh: 5,709 J kumulatif:229,762 (84.4%)  J kes baru:2,998  J kumulatif:272,163  J kes aktif:41,396  Kes local:2,991 (1,578WN-52.8%, 1,413BWN) Kes import:7(4BWN)   J kematian:22 (21WN) J kumulatif:1,005 (0.37%)  J kes di ICU:231 Pesakit Intubated:115 '

In [ ]:
tweets_df_bk = tweets_df.copy()

In [ ]:
tweets_df = tweets_df_bk.copy()

In [25]:
lst_df = []

for i in range(len(tweets_df)):
  text = tweets_df.loc[i,'text']
  
  lst_tweet = re.findall(r"[0-9]+[,.]?[0-9]*[,.]?[0-9]*", text)
  lst_col0_8 = lst_tweet[3:12]        # elemnt 4 until 12  # total 9  
  lst_col17_20 = lst_tweet[-4:]

  importcase = re.search(r"(import)(.*?)(J)", text, re.IGNORECASE).group(0)                     # we only know something is in it
  import_total = re.search(r"[0-9]+", importcase).group(0)                                      # get the first number  # total 
  if import_total != '0':                                                                       # if not 0 
    import_wn_re = re.search(r"([0-9]+)(?:\s?)(?:WN)", importcase, re.IGNORECASE)               # get WN value
    import_bwn_re = re.search(r"([0-9]+)(?:\s?)(?:BWN)", importcase, re.IGNORECASE)             # get BWN value
    if import_wn_re is not None:                                                                 
      import_wn = import_wn_re.group(1)
      if import_bwn_re is None:
        import_bwn = str(int(import_total) - int(import_wn))
    if import_bwn_re is not None:
      import_bwn = import_bwn_re.group(1)
      if import_wn_re is None:
        import_wn = str(int(import_total) - int(import_bwn)) 
    if import_wn_re == import_bwn_re == None:                                                   # if both dont exist, assign empty string
      import_wn = import_bwn = '' 
  else: import_total = import_wn = import_bwn = '0'


  maticase = re.search(r"(mati|fataliti)(.*?)(J)", text, re.IGNORECASE).group(0)                 # we only know something is in it
  mati_total = re.search(r"[0-9]+", maticase).group(0)                                           # get the first number  # total 
  if import_total != '0':                                                                        # if not 0 
    mati_wn_re = re.search(r"([0-9]+)(?:\s?)(?:WN)", maticase, re.IGNORECASE)
    mati_bwn_re = re.search(r"([0-9]+)(?:\s?)(?:BWN)", maticase, re.IGNORECASE)
    if mati_wn_re is not None:
      mati_wn = mati_wn_re.group(1)
      if mati_bwn_re is None:
        mati_bwn = str(int(mati_total) - int(mati_wn))
    if mati_bwn_re is not None:
      mati_bwn = mati_bwn_re.group(1)
      if mati_wn_re is None:
        mati_wn = str(int(mati_total) - int(mati_bwn)) 
    if mati_wn_re == mati_bwn_re == None:
      mati_wn = mati_bwn = '' 
  else: mati_total = mati_wn = mati_bwn = '0'

  lst_col9_15 = [import_total, import_wn, import_bwn, mati_total, mati_wn, mati_bwn]

  lst_cols = lst_col0_8 + lst_col9_15 + lst_col17_20
  lst_df.append(lst_cols)

In [40]:
# cols = ['Recovered', 'Cumulative recovered', 'Cumulative recovered  (% of all cases)', 'New case', 
#         'Cumulative cases', 'Active case', 'Local case', 'Local case (Citizen)', 'Local case (Non-citizen)', 
#         'Imported case', 'Imported case (Citizen)', 'Imported case (Non-citizen)', 'Death', 'Death (Citizen)', 
#         'Death (Non-citizen)', 'Cumulative deaths', 'Cumulative deaths (% of all cases)', 
#         'ICU', 'Intubated']

# 'cumul_people_tested', 'people_tested', 'case_avg', 'pos_rate', 'pos_rate_avg'

cols = ['recovered', 'cumul_recovered', 'cumul_recovered_perc', 'case', 'cumul_case', 
        'case_active', 'case_local', 'case_local_wn', 'case_local_bwn', 'case_import', 
        'case_import_wn', 'case_import_bwn', 'death', 'death_wn', 'death_bwn', 
        'cumul_death', 'cumul_death_perc', 'icu', 'intubated']
        
df = pd.DataFrame(lst_df, columns=cols)      # 19 cols passed
df.insert(0, 'date', tweets_df['date'])      # + 1 col date
df['tweet_id'] = tweets_df['tweet_id']       # + 1 col tweet_id
df['link'] = tweets_df['link']               # + 1 col link

In [41]:
df

,date,recovered,cumul_recovered,cumul_recovered_perc,case,cumul_case,case_active,case_local,case_local_wn,case_local_bwn,case_import,case_import_wn,case_import_bwn,death,death_wn,death_bwn,cumul_death,cumul_death_perc,icu,intubated,tweet_id,link
0,2021-02-15,"4,521","218,335",81.9,"2,176","266,445","47,135","2,175","1,618",74.4,1,1,0,10,10,0,975,0.37,260,112,1361250755588485123,t.co/Yhr88R9Shv
1,2021-02-16,"5,718","224,053",83.24,"2,720","269,165","44,129","2,720","1,391",51.1,0,0,0,0,0,0,983,0.37,253,118,1361632748994060288,t.co/INHqORa9ja
2,2021-02-17,"5,709","229,762",84.4,"2,998","272,163","41,396","2,991","1,578",52.8,7,3,4,22,21,1,"1,005",0.37,231,115,1361980739235893251,t.co/rLeKuYlA6L
3,2021-02-18,"5,320","235,082",85.5,"2,712","274,875","38,763","2,708","1,588",58.6,4,3,1,25,21,4,"1,030",0.37,227,103,1362345745727315969,t.co/eYXvcZ25Fb
4,2021-02-19,"4,889","239,971",86.4,"2,936","277,811","36,797","2,919","1,572",53.9,17,4,13,13,11,2,"1,043",0.38,220,104,1362703258456711169,t.co/d7BDBLGSTv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,2021-07-21,"7,902","806,857",84.8,"11,985","951,884","137,587","11,978","10,445","1,533",7,5,2,199,173,26,"7,440",0.78,927,459,1417798072185212929,t.co/wVhqfvmO24
154,2021-07-22,"8,436","815,293",84.5,"13,034","964,918","142,051","13,009","11,446","1,563",25,19,6,134,119,15,"7,574",0.79,938,459,1418169018712686595,t.co/nn6g2PySnt
155,2021-07-23,"10,094","825,387",84.18,"15,573","980,491","147,386","15,541","13,395","2,146",32,12,20,144,132,12,"7,718",0.79,939,456,1418525491049361411,t.co/94rlWVXKBm
156,2021-07-24,"9,471","834,858",83.79,"15,902","996,393","153,633","15,876","12,842","3,034",26,7,19,184,157,27,"7,902",0.79,950,468,1418886465052946434,t.co/joYSRdlcF7


## Concat df

In [47]:
df1 = pd.read_csv('tweets_df2_result.csv')
df1['date'] = pd.to_datetime(df1['date'])

In [48]:
result_df = pd.concat([df1, df])
result_df

,date,recovered,cumul_recovered,cumul_recovered_perc,case,cumul_case,case_active,case_local,case_local_wn,case_local_bwn,case_import,case_import_wn,case_import_bwn,death,death_wn,death_bwn,cumul_death,cumul_death_perc,icu,intubated,tweet_id,link
0,2020-09-07,9,9124,96.5,62,9459,207,56,30,26,6,2,4,0,NaN,NaN,128,1.35,"6,",4,1302890281642074112,t.co/k4C9NzPuRt
1,2020-09-08,12,9136,96.6,100,9559,295,85,NaN,NaN,15,NaN,NaN,0,NaN,NaN,128,1.35,7,4,1303273872012140544,t.co/xsTCM7mYCE
2,2020-09-09,7,9143,95.4,24,9583,317,18,NaN,NaN,6,NaN,NaN,0,NaN,NaN,128,1.35,7,4,1303621756515614721,t.co/bEuxIL2va5
3,2020-09-11,14,9181,93.59,182,9810,501,181,90,91,1,NaN,NaN,0,NaN,NaN,128,1.3,9,5,1304350471872684032,t.co/MA0t8Xdd9L
4,2020-09-12,8,9189,93.59,58,9868,551,53,NaN,NaN,5,3,2,0,NaN,NaN,128,1.3,9,5,1304706214794723329,t.co/PxmVzcehaZ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,2021-07-21,"7,902","806,857",84.8,"11,985","951,884","137,587","11,978","10,445","1,533",7,5,2,199,173,26,"7,440",0.78,927,459,1417798072185212929,t.co/wVhqfvmO24
154,2021-07-22,"8,436","815,293",84.5,"13,034","964,918","142,051","13,009","11,446","1,563",25,19,6,134,119,15,"7,574",0.79,938,459,1418169018712686595,t.co/nn6g2PySnt
155,2021-07-23,"10,094","825,387",84.18,"15,573","980,491","147,386","15,541","13,395","2,146",32,12,20,144,132,12,"7,718",0.79,939,456,1418525491049361411,t.co/94rlWVXKBm
156,2021-07-24,"9,471","834,858",83.79,"15,902","996,393","153,633","15,876","12,842","3,034",26,7,19,184,157,27,"7,902",0.79,950,468,1418886465052946434,t.co/joYSRdlcF7


In [49]:
result_df.to_csv(r'malaysia covid dataset 2020-2021.csv', index = False)